In [ ]:
class ExtraS1S2Properties(hax.minitrees.TreeMaker):
    '''
    
    '''
    __version__ = '0.0.0'
    extra_branches = ['peaks.*', 'interactions*']
    
    
    def extract_data(self, event):
        # ================ PEAK PROPERTIES =========================
        # Extract data for these types
        peak_types = ['s1', 's2']
        
        # Peak_properties (will be loaded for s1_0, s1_1, s2_0, s2_1)
        peak_properties = ['n_saturated_channels']

        result_vars = [_pt + '_' + _pp for _pt in peak_types for _pp in peak_properties ]
        result = {k: float('nan') for k in result_vars}
        
        s1s = []
        s2s = []
        n_interactions = len(event.interactions)
        
        # For the first two interactions, take their s1 and s2 index, if they exist
        for interaction in event.interactions:
            # Get s1 and s2 index
            s1s.append(interaction.s1)
            s2s.append(interaction.s2)

        
        # ================= LOAD PEAK DATA ==========================
        # Loop over the two peak types
        for pt in peak_types:
            # Load index list
            if pt == 's1':
                peak_list = s1s
            elif pt == 's2':
                peak_list = s2s
            else:
                raise NotImplementedError('Peak types s1 and s2 supported only!')
            
            # Now, for this peak type, loop over peak_indices and get peak pos
            if len(peak_list) == 0:
                # If there are not enough peaks of this type, do not try to find properties
                continue
            else:
                peak_pos = int(peak_list[0])

                # Time to extract data
                peak_name = pt + '_' # Example: s1_0_

                # All standard properties
                for prop in peak_properties:
                    result[peak_name + prop] = getattr(event.peaks[peak_pos], prop)
        
        return result

In [ ]:
class XAMSProperties(hax.minitrees.TreeMaker):
    '''
    Get properties for XAMS
    
    '''
    __version__ = '0.1.0'
    
    extra_branches = ['peaks.*', 'peaks.range_area_decile*', 
                      'interactions*']
    
    def extract_data(self, event):
        
        peak_types = ['s1', 's2']
        peak_properties = ['area_fraction_top', 'area', 'center_time', 'n_saturated_channels']
        extra_properties = ['range_50p_area', 'range_70p_area']
        all_properties = peak_properties + extra_properties
        result_vars = [_pt + '_' + _pp for _pt in peak_types for _pp in all_properties ]
        result = {k: float('nan') for k in result_vars}
        
        interaction_properties = ['drift_time']
        for _prop in interaction_properties:
            result[_prop] = float('nan')
            
        s1s = []
        s2s = []
        
        for peak in event.peaks:
            if peak.type == 's1':
                s1s.append(peak)
            if peak.type == 's2':
                s2s.append(peak)
        
        # Reverse-sort by area
        s1s = sorted(s1s, key = lambda peak: - peak.area)
        s2s = sorted(s2s, key = lambda peak: - peak.area)

        
        # Now load properies
        result['n_s1s'] = len(event.s1s)
        result['n_s2s'] = len(event.s2s)
        
        
        for peak_name, peak_list in zip(['s1', 's2'], [s1s, s2s]):
            if len(peak_list) == 0:
                continue
            # First peak in list sorted by area
            peak = peak_list[0]
            for _prop in peak_properties:
                result[peak_name + '_' + _prop] = getattr(peak, _prop)
            for i in range(10):
                if ('range_%d0p_area' % i) in extra_properties:
                    result[peak_name + '_' + 'range_%d0p_area' % i] = list(peak.range_area_decile)[i]
        
        if len(s2s) > 1:
            result['largest_other_s2'] = getattr(s2s[1], 'area')
        else:
            result['largest_other_s2'] = 0.
            
        result['drift_time'] = result['s2_center_time'] - result['s1_center_time']
        



#             for peak_type, peak_index in zip(peak_types, [s1,s2]):
#                 for _prop in peak_properties:
#                     result[peak_name + '_' + _prop] = getattr(event.peaks[peak_index], _prop)

#                 if 'range_50p_area' in extra_properties:
#                     result[peak_name + '_' + 'range_50p_area'] = list(event.peaks[peak_pos].range_area_decile)[5]
        
        return result